<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Template

In [171]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [172]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [174]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-28 15:06:37,010 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-28 15:06:37,025 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-28 15:06:37,042 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:140227481359008]
2024-06-28 15:06:37,043 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2024-06-28 15:06:37,048 - INFO - RenewingAuthorizer will start by using access_token with hash "22698e60f6bccb0a365acf368e682f156294bf464a7405d9135abeee27d19861"
2024-06-28 15:06:37,049 - INFO - Executing token refresh without client credentials
2024-06-28 15:06:37,050 - INFO - Fetching new token from Globus Auth
2024-06-28 15:06:37,621 - INFO - request done (success)
2024-06-28 15:06:37,622 - INFO - RenewingAuthorizer.

You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [175]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C61G" # rid



In [176]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-28 15:06:38,028 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-28 15:06:38,029 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [177]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-28 15:06:42,017 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json] transfer successful. 0.68 KB transferred. Elapsed time: 0:00:00.000115.
2024-06-28 15:06:42,018 - INFO - Verifying SHA256 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json]
2024-06-28 15:06:42,055 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_data_analysis',
    'rid': '2-BDAC'}],
  'Execution_Asset_Type': [{'name': 'Multimodal_Analysis', 'rid': '2-BDAE'}]},
 'execution_rid': '2-C8TE',
 'workflow_rid': '2-BDAG',
 'bag_paths': [PosixPath('/data/2-7P5P_1c2dda3d961d50c2b541eb05ae87ad35ebe02b04402894b42dc5ed594cef944c/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json')}

In [178]:
# View data

subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv')
subject

observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv')
observation

clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv')
clinic

observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv')
observation_clinic_asso # association table between observation table and clinic record table

icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv')
icd10

icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv')
icd10_asso # association table between clinic record table and ICD10 code

# report = pd.read_csv(configuration_records.bag_paths[0]/'data/Report.csv')
# report

# RNFL_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/RNFL_OCR.csv')
# RNFL_OCR

# HVF_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/HVF_OCR.csv')
# HVF_OCR

print()

In [179]:
subject


,RID,RCT,RMT,RCB,RMB,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Subject_Ethnicity,Subject_hypertension,Subject_insulin_dependent,Subject_pregnant,Subject_cataract,Subject_maculopathy,Subject_other,Subject_image_quality,site_mrn,Consultant
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-7KTW,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,3,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-7KTY,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,4,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-7KV0,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,5,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2-7P5C,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1195,NaN,NaN,NaN,NaN,...,69WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,2-7P5E,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1196,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2-7P5G,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1197,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,69WE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
observation

,RID,RCT,RMT,RCB,RMB,Observation_ID,Subject,Date_of_Encounter
0,2-7RGP,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,2-7KTR,2020-08-05 00:00:00+00
1,2-7RGR,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,2-7KTR,2020-10-22 00:00:00+00
2,2-7RGT,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,659,2-7KTT,2023-04-18 00:00:00+00
3,2-7RGW,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,660,2-7KTT,2023-04-19 00:00:00+00
4,2-7RGY,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,658,2-7KTT,2022-12-20 00:00:00+00
...,...,...,...,...,...,...,...,...
2541,2-7XFG,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,471,2-7P5J,2022-09-14 00:00:00+00
2542,2-7XFJ,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,470,2-7P5J,2022-04-20 00:00:00+00
2543,2-7XFM,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,474,2-7P5M,2023-04-19 00:00:00+00
2544,2-7XFP,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,473,2-7P5M,2022-10-19 00:00:00+00


In [181]:
# Merge Subject with Observation
subject_observation = pd.merge(subject, observation, left_on='RID', right_on='Subject', how='inner', suffixes=('_subject_df', '_observation_df'))
subject_observation

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,site_mrn,Consultant,RID_observation_df,RCT_observation_df,RMT_observation_df,RCB_observation_df,RMB_observation_df,Observation_ID,Subject,Date_of_Encounter
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGP,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,2-7KTR,2020-08-05 00:00:00+00
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGR,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,2-7KTR,2020-10-22 00:00:00+00
2,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGT,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,659,2-7KTT,2023-04-18 00:00:00+00
3,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGW,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,660,2-7KTT,2023-04-19 00:00:00+00
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGY,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,658,2-7KTT,2022-12-20 00:00:00+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFG,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,471,2-7P5J,2022-09-14 00:00:00+00
2542,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFJ,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,470,2-7P5J,2022-04-20 00:00:00+00
2543,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFM,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,474,2-7P5M,2023-04-19 00:00:00+00
2544,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFP,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,473,2-7P5M,2022-10-19 00:00:00+0

In [182]:
# Merge with Observation_Clinical_Association
subject_obs_clinic = pd.merge(subject_observation, observation_clinic_asso, left_on= 'RID_observation_df', right_on='Observation', suffixes=('_subject_observation_df', "_observation_clinic_asso_df"), how='inner')

subject_obs_clinic

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Observation_ID,Subject,Date_of_Encounter,RID,RCT,RMT,RCB,RMB,Clinical_Records,Observation
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,1,2-7KTR,2020-08-05 00:00:00+00,2-8QZY,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT4,2-7RGP
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,1,2-7KTR,2020-08-05 00:00:00+00,2-8R00,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7RGP
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,2,2-7KTR,2020-10-22 00:00:00+00,2-8R02,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT8,2-7RGR
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,2,2-7KTR,2020-10-22 00:00:00+00,2-8R04,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7RGR
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,659,2-7KTT,2023-04-18 00:00:00+00,2-8R06,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTC,2-7RGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,474,2-7P5M,2023-04-19 00:00:00+00,2-91GR,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868W,2-7XFM
4878,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,473,2-7P5M,2022-10-19 00:00:00+00,2-91GT,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XFP
4879,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,473,2-7P5M,2022-10-19 00:00:00+00,2-91GW,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8690,2-7XFP
4880,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,472,2-7P5M,2022-04-20 00:00:00+00,2-91GY,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XFR


In [183]:
clinic

,RID,RCT,RMT,RCB,RMB,Date_of_Encounter,LogMAR_VA,Visual_Acuity_Numerator,IOP,Refractive_Error,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label
0,2-7XT4,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.14,20.0,13.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M
1,2-7XT6,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.22,20.0,14.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M
2,2-7XT8,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,14.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M
3,2-7XTA,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,13.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M
4,2-7XTC,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-18 00:00:00,0.18,20.0,9.0,-0.5,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332,2-868W,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-19 00:00:00,0.18,20.0,19.0,NaN,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J
4333,2-868Y,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.18,20.0,16.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J
4334,2-8690,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.22,20.0,16.0,NaN,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J
4335,2-8692,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-04-20 00:00:00,0.18,20.0,14.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J


In [184]:
clinic['Condition_Label'].unique()

array(['2-C60M', '2-C60J', nan, '2-C60P'], dtype=object)

In [185]:
import numpy as np

# Create the first new column with condition names
condition_map = {
    '2-C60J': 'GS',
    '2-C60M': 'POAG',
    '2-C60P': 'PACG'
}

clinic['Condition_Name'] = clinic['Condition_Label'].map(condition_map)

# Create the second new column with Glaucoma classification
def classify_glaucoma(condition):
    if condition in ['POAG', 'PACG']:
        return 'Glaucoma'
    elif condition == 'GS':
        return 'Glaucoma suspects'
    else:
        return np.nan

clinic['Glaucoma_Classification'] = clinic['Condition_Name'].apply(classify_glaucoma)

clinic

,RID,RCT,RMT,RCB,RMB,Date_of_Encounter,LogMAR_VA,Visual_Acuity_Numerator,IOP,Refractive_Error,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7XT4,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.14,20.0,13.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7XT6,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.22,20.0,14.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7XT8,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,14.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7XTA,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,13.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7XTC,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-18 00:00:00,0.18,20.0,9.0,-0.5,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332,2-868W,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-19 00:00:00,0.18,20.0,19.0,NaN,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4333,2-868Y,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.18,20.0,16.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4334,2-8690,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.22,20.0,16.0,NaN,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4335,2-8692,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-04-20 00:00:00,0.18,20.0,14.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [186]:
clinic['Glaucoma_Classification'].unique()

array(['Glaucoma', 'Glaucoma suspects', nan], dtype=object)

In [187]:
subject_obs_clinic_data = pd.merge(subject_obs_clinic, clinic, left_on='Clinical_Records', right_on='RID', suffixes=("_subject_obs_clinic_df", "_clinic_df"), how='inner')




subject_obs_clinic_data

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4878,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4879,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4880,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [188]:
subject_obs_clinic_data['Glaucoma_Classification'].unique()

array(['Glaucoma', 'Glaucoma suspects', nan], dtype=object)

In [189]:
subject_obs_clinic_data['RID_subject_df'].nunique()

1157

In [190]:

# Assuming subject_obs_clinic_data is your final merged dataframe
unique_subject_counts = subject_obs_clinic_data.groupby('Glaucoma_Classification')['RID_subject_df'].nunique()

# Print the unique subject counts for each classification
print(unique_subject_counts)

Glaucoma_Classification
Glaucoma             708
Glaucoma suspects    445
Name: RID_subject_df, dtype: int64


In [191]:

# Assuming subject_obs_clinic_data is your final merged dataframe
unique_subject_counts = subject_obs_clinic_data.groupby('Glaucoma_Classification')['Subject_ID'].nunique()

# Print the unique subject counts for each classification
print(unique_subject_counts)

Glaucoma_Classification
Glaucoma             708
Glaucoma suspects    445
Name: Subject_ID, dtype: int64


# GS 

In [192]:
# All code for creating GS suspect entries

subject_obs_clinic_data_gs = subject_obs_clinic_data.copy(deep=True)

subject_obs_clinic_data_gs

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4878,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4879,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4880,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [193]:
subject_obs_clinic_data_gs = subject_obs_clinic_data_gs[
    (subject_obs_clinic_data_gs['IOP'] >= 0) &
    (subject_obs_clinic_data_gs['IOP'] <= 21) &
    (subject_obs_clinic_data_gs['Condition_Name'] == 'GS')
]

subject_obs_clinic_data_gs

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
10,2-7KTW,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,3,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Open angle with borderline findings, low risk,...",NaN,2SK2,16889,2-C60J,GS,Glaucoma suspects
11,2-7KTW,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,3,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Open angle with borderline findings, low risk,...",NaN,2SK2,16889,2-C60J,GS,Glaucoma suspects
12,2-7KTY,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,4,NaN,NaN,NaN,NaN,...,NaN,0.70,"(B)D30s1+ inf, D30s1+ elsewhere","Unspecified blepharitis unspecified eye, unspe...",NaN,2SK0,30294,2-C60J,GS,Glaucoma suspects
13,2-7KTY,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,4,NaN,NaN,NaN,NaN,...,NaN,0.85,"(B)D30s1+ inf, D30s1+ elsewhere","Unspecified blepharitis unspecified eye, unspe...",NaN,2SK2,30295,2-C60J,GS,Glaucoma suspects
14,2-7KTY,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,4,NaN,NaN,NaN,NaN,...,532.0,0.70,"(B)D30s1+ inf, D30s1+ elsewhere","Presbyopia_ Other disorders of optic disc, bil...",Hossein Ameri,2SK0,14510,2-C60J,GS,Glaucoma suspects
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4878,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4879,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4880,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [194]:
icd10_gs = icd10.copy(deep=True)

icd10_gs

,RID,RCT,RMT,RCB,RMB,ID,URI,Name,Description,Synonyms,ICD10
0,2-7XFT,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFT,/id/2-7XFT,H40.002,"Preglaucoma, unspecified, left eye",{},H40.002
1,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003
2,2-7XFY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFY,/id/2-7XFY,H40.009,"Preglaucoma, unspecified, unspecified eye",{},H40.009
3,2-7XG0,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG0,/id/2-7XG0,H40.012,"Open angle with borderline findings, low risk,...",{},H40.012
4,2-7XG2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG2,/id/2-7XG2,H40.013,"Open angle with borderline findings, low risk,...",{},H40.013
...,...,...,...,...,...,...,...,...,...,...,...
114,2-7XSM,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSM,/id/2-7XSM,H25.11,"Age-related nuclear cataract, right eye",{},H25.11
115,2-7XST,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XST,/id/2-7XST,H52.11,"Myopia, right eye",{},H52.11
116,2-7XSW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSW,/id/2-7XSW,H52.01,"Hypermetropia, right eye",{},H52.01
117,2-7XSY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSY,/id/2-7XSY,H40.211,"Acute angle-closure glaucoma, right eye",{},H40.211


In [195]:
icd10_gs = icd10_gs[icd10_gs['ICD10'].isin(['H40.003', 'H40.013', 'H40.023'])]

icd10_gs 

,RID,RCT,RMT,RCB,RMB,ID,URI,Name,Description,Synonyms,ICD10
1,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003
4,2-7XG2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG2,/id/2-7XG2,H40.013,"Open angle with borderline findings, low risk,...",{},H40.013
7,2-7XG8,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG8,/id/2-7XG8,H40.023,"Open angle with borderline findings, high risk...",{},H40.023


In [196]:
icd10_asso_gs = icd10_asso.copy(deep=True)

icd10_asso_gs

,RID,RCT,RMT,RCB,RMB,Clinical_Records,ICD10_Eye
0,2-8696,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7XHT
1,2-8698,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7XPE
2,2-869A,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7XHT
3,2-869C,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7XPE
4,2-869E,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTE,2-7XHT
...,...,...,...,...,...,...,...
9063,2-8QZM,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XG8
9064,2-8QZP,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XNY
9065,2-8QZR,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XFW
9066,2-8QZT,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XG8


In [197]:
# Merge the dataframes
icd10_and_icd10_asso_merged_df = pd.merge(icd10_gs, icd10_asso_gs, left_on='RID', right_on='ICD10_Eye', how='left',  suffixes=("_icd10_gs_df", "_icd10_asso_gs_df"))


icd10_and_icd10_asso_merged_df

,RID_icd10_gs_df,RCT_icd10_gs_df,RMT_icd10_gs_df,RCB_icd10_gs_df,RMB_icd10_gs_df,ID,URI,Name,Description,Synonyms,ICD10,RID_icd10_asso_gs_df,RCT_icd10_asso_gs_df,RMT_icd10_asso_gs_df,RCB_icd10_asso_gs_df,RMB_icd10_asso_gs_df,Clinical_Records,ICD10_Eye
0,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86A8,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XVJ,2-7XFW
1,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86B6,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XWE,2-7XFW
2,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86DJ,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XZ6,2-7XFW
3,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86E8,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XZY,2-7XFW
4,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86EC,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7Y02,2-7XFW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,2-7XG8,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG8,/id/2-7XG8,H40.023,"Open angle with borderline findings, high risk...",{},H40.023,2-8QWT,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-866J,2-7XG8
1271,2-7XG8,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG8,/id/2-7XG8,H40.023,"Open angle with borderline findings, high risk...",{},H40.023,2-8QWY,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-866P,2-7XG8
1272,2-7XG8,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG8,/id/2-7XG8,H40.023,"Open angle with borderline findings, high risk...",{},H40.023,2-8QZG,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868T,2-7XG8
1273,2-7XG8,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-

In [198]:
# Merging the dataframes based on Clinical_ID and Clinical_Records
final_merged_df_gs = pd.merge(subject_obs_clinic_data_gs, icd10_and_icd10_asso_merged_df, 
                           left_on='RID_clinic_df', right_on='Clinical_Records', how='inner')

# Display the result
# print(final_merged_df)

final_merged_df_gs

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Description,Synonyms,ICD10,RID_icd10_asso_gs_df,RCT_icd10_asso_gs_df,RMT_icd10_asso_gs_df,RCB_icd10_asso_gs_df,RMB_icd10_asso_gs_df,Clinical_Records_y,ICD10_Eye
0,2-7KV0,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,5,NaN,NaN,NaN,NaN,...,"Open angle with borderline findings, low risk,...",{},H40.013,2-8F56,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XV2,2-7XG2
1,2-7KV0,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,5,NaN,NaN,NaN,NaN,...,"Open angle with borderline findings, low risk,...",{},H40.013,2-86A2,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XV4,2-7XG2
2,2-7KV4,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,7,NaN,NaN,NaN,NaN,...,"Preglaucoma, unspecified, bilateral",{},H40.003,2-8F5E,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XVE,2-7XFW
3,2-7KV4,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,7,NaN,NaN,NaN,NaN,...,"Open angle with borderline findings, high risk...",{},H40.023,2-8F5G,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XVE,2-7XG8
4,2-7KV4,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,7,NaN,NaN,NaN,NaN,...,"Preglaucoma, unspecified, bilateral",{},H40.003,2-86A8,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XVJ,2-7XFW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,"Open angle with borderline findings, high risk...",{},H40.023,2-8F44,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8690,2-7XG8
1280,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,"Preglaucoma, unspecified, bilateral",{},H40.003,2-8QZR,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XFW
1281,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,"Open angle with borderline findings, high risk...",{},H40.023,2-8QZT,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XG8
1282,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3

In [199]:
final_merged_df_gs.Clinical_Records_y.nunique()

932

In [200]:
final_merged_df_gs.Subject_ID

0          5
1          5
2          7
3          7
4          7
        ... 
1279    1199
1280    1199
1281    1199
1282    1199
1283    1199
Name: Subject_ID, Length: 1284, dtype: int64

In [201]:
final_merged_df_gs.to_csv("final_merged_df_gs.csv", index=False)

## Another way to test GS

In [202]:
import pandas as pd
import numpy as np

# Load data
subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv')
observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv')
clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv')
observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv')
icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv')
icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv')

# Step 1: Apply initial filters to clinical records
clinic_filtered = clinic[
    (clinic['IOP'] >= 0) & 
    (clinic['IOP'] <= 21) & 
    (clinic['Condition_Label'] == '2-C60J')
]
print(f"Number of records with IOP 0-21 and GS condition: {len(clinic_filtered)}")

# Step 2: Prepare ICD-10 data
valid_icd10 = ['H40.003', 'H40.013', 'H40.023']
icd10_filtered = icd10[icd10['ICD10'].isin(valid_icd10)]
icd10_asso_filtered = icd10_asso[icd10_asso['ICD10_Eye'].isin(icd10_filtered['RID'])]

# Step 3: Merge clinical records with ICD-10 associations
merged_df = pd.merge(clinic_filtered, icd10_asso_filtered, 
                     left_on='RID', right_on='Clinical_Records', 
                     how='inner')

# Step 4: Add ICD-10 code to merged dataframe
merged_df = pd.merge(merged_df, icd10_filtered[['RID', 'ICD10']], 
                     left_on='ICD10_Eye', right_on='RID', 
                     suffixes=('', '_icd10'))

# Step 5: Group by Clinical_Records and aggregate ICD10 codes
grouped_df = merged_df.groupby('Clinical_Records').agg({
    'RID': 'first',
    'IOP': 'first',
    'Condition_Label': 'first',
    'ICD10': lambda x: ','.join(sorted(set(x)))
}).reset_index()

print(f"Number of unique clinical records: {len(grouped_df)}")

# Function to filter by specific ICD-10 codes
def filter_by_icd10(df, codes):
    return df[df['ICD10'].apply(lambda x: any(code in x.split(',') for code in codes))]

# Individual ICD-10 code filters
for code in valid_icd10:
    filtered = filter_by_icd10(grouped_df, [code])
    print(f"Number of unique clinical records for {code}: {len(filtered)}")

# Combined H40.003 and H40.013
combined_filtered = filter_by_icd10(grouped_df, ['H40.003', 'H40.013'])
print(f"Number of unique clinical records for H40.003 and H40.013 combined: {len(combined_filtered)}")

# Combined H40.023 and H40.013
combined_filtered = filter_by_icd10(grouped_df, ['H40.023', 'H40.013'])
print(f"Number of unique clinical records for H40.023 and H40.013 combined: {len(combined_filtered)}")

# Combined H40.003 and H40.023
combined_filtered = filter_by_icd10(grouped_df, ['H40.003', 'H40.023'])
print(f"Number of unique clinical records for H40.003 and H40.023 combined: {len(combined_filtered)}")

# All ICD-10 codes combined
all_filtered = filter_by_icd10(grouped_df, valid_icd10)
print(f"Total number of unique clinical records with any of the ICD-10 codes: {len(all_filtered)}")

# Display distribution of ICD-10 codes
print("\nDistribution of ICD-10 codes:")
icd10_distribution = grouped_df['ICD10'].apply(lambda x: x.split(',')).explode().value_counts()
print(icd10_distribution)

# Display a few rows to verify the result
print("\nSample of final dataframe:")
print(grouped_df.head())

print(f"\nTotal rows in final dataframe: {len(grouped_df)}")

# Optional: Save the final dataframe to a CSV file
# grouped_df.to_csv('final_clinical_icd10_data.csv', index=False)
print("Final dataframe saved to 'final_clinical_icd10_data.csv'")

Number of records with IOP 0-21 and GS condition: 1222
Number of unique clinical records: 932
Number of unique clinical records for H40.003: 354
Number of unique clinical records for H40.013: 320
Number of unique clinical records for H40.023: 465
Number of unique clinical records for H40.003 and H40.013 combined: 586
Number of unique clinical records for H40.023 and H40.013 combined: 785
Number of unique clinical records for H40.003 and H40.023 combined: 700
Total number of unique clinical records with any of the ICD-10 codes: 932

Distribution of ICD-10 codes:
ICD10
H40.023    465
H40.003    354
H40.013    320
Name: count, dtype: int64

Sample of final dataframe:
  Clinical_Records     RID   IOP Condition_Label            ICD10
0           2-7XV2  2-7XG2  16.0          2-C60J          H40.013
1           2-7XV4  2-7XG2  16.0          2-C60J          H40.013
2           2-7XVE  2-7XFW  20.0          2-C60J  H40.003,H40.023
3           2-7XVJ  2-7XFW  19.0          2-C60J  H40.003,H40.0

In [203]:
set(grouped_df.Clinical_Records) == set(final_merged_df_gs.Clinical_Records_y)

True

# Glaucoma

In [204]:
# All code for creating Glaucoma entries

subject_obs_clinic_data_g = subject_obs_clinic_data.copy(deep=True)

subject_obs_clinic_data_g


,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4878,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4879,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4880,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [205]:

subject_obs_clinic_data_g = subject_obs_clinic_data_g[
    (subject_obs_clinic_data_g['Condition_Name'] == 'POAG')
]

subject_obs_clinic_data_g

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4867,2-7P5E,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1196,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Type 2 diabetes mellitus with proliferative di...,Andrew Moshfeghi,2SK2,57458,2-C60M,POAG,Glaucoma
4872,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Low-tension glaucoma, bilateral, mild stage_ A...",Brian Song,2SK0,34631,2-C60M,POAG,Glaucoma
4873,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,0.75,NaN,"Low-tension glaucoma, bilateral, mild stage_ A...",Brian Song,2SK2,34632,2-C60M,POAG,Glaucoma
4874,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Vitreous degeneration, bilateral_ Other specif...",Brian Song,2SK0,34629,2-C60M,POAG,Glaucoma


In [206]:


icd10_g = icd10.copy(deep=True)

icd10_g

,RID,RCT,RMT,RCB,RMB,ID,URI,Name,Description,Synonyms,ICD10
0,2-7XFT,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFT,/id/2-7XFT,H40.002,"Preglaucoma, unspecified, left eye",{},H40.002
1,2-7XFW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFW,/id/2-7XFW,H40.003,"Preglaucoma, unspecified, bilateral",{},H40.003
2,2-7XFY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XFY,/id/2-7XFY,H40.009,"Preglaucoma, unspecified, unspecified eye",{},H40.009
3,2-7XG0,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG0,/id/2-7XG0,H40.012,"Open angle with borderline findings, low risk,...",{},H40.012
4,2-7XG2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XG2,/id/2-7XG2,H40.013,"Open angle with borderline findings, low risk,...",{},H40.013
...,...,...,...,...,...,...,...,...,...,...,...
114,2-7XSM,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSM,/id/2-7XSM,H25.11,"Age-related nuclear cataract, right eye",{},H25.11
115,2-7XST,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XST,/id/2-7XST,H52.11,"Myopia, right eye",{},H52.11
116,2-7XSW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSW,/id/2-7XSW,H52.01,"Hypermetropia, right eye",{},H52.01
117,2-7XSY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XSY,/id/2-7XSY,H40.211,"Acute angle-closure glaucoma, right eye",{},H40.211


In [207]:



icd10_g = icd10_g[icd10_g['ICD10'].isin([
    "H40.1130",
    "H40.1131",
    "H40.1132",
    "H40.1133",
    "H40.1134",
    "H40.1231",
    "H40.1232",
    "H40.1233",
    "H40.1234"
])]

icd10_g 

,RID,RCT,RMT,RCB,RMB,ID,URI,Name,Description,Synonyms,ICD10
28,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130
29,2-7XHR,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHR,/id/2-7XHR,H40.1131,"Primary open-angle glaucoma, bilateral, mild s...",{},H40.1131
30,2-7XHT,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHT,/id/2-7XHT,H40.1132,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132
31,2-7XHW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHW,/id/2-7XHW,H40.1133,"Primary open-angle glaucoma, bilateral, severe...",{},H40.1133
32,2-7XHY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHY,/id/2-7XHY,H40.1134,"Primary open-angle glaucoma, bilateral, indete...",{},H40.1134
42,2-7XJW,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XJW,/id/2-7XJW,H40.1231,"Low-tension glaucoma, bilateral, mild stage",{},H40.1231
43,2-7XJY,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XJY,/id/2-7XJY,H40.1232,"Low-tension glaucoma, bilateral, moderate stage",{},H40.1232
44,2-7XK0,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XK0,/id/2-7XK0,H40.1233,"Low-tension glaucoma, bilateral, severe stage",{},H40.1233
45,2-7XK2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XK2,/id/2-7XK2,H40.1234,"Low-tension glaucoma, bilateral, indeterminate...",{},H40.1234


In [208]:



icd10_asso_g = icd10_asso.copy(deep=True)

icd10_asso_g

,RID,RCT,RMT,RCB,RMB,Clinical_Records,ICD10_Eye
0,2-8696,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7XHT
1,2-8698,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7XPE
2,2-869A,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7XHT
3,2-869C,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7XPE
4,2-869E,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTE,2-7XHT
...,...,...,...,...,...,...,...
9063,2-8QZM,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XG8
9064,2-8QZP,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XNY
9065,2-8QZR,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XFW
9066,2-8QZT,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XG8


In [209]:



# Merge the dataframes
icd10_and_icd10_asso_merged_df = pd.merge(icd10_g, icd10_asso_g, left_on='RID', right_on='ICD10_Eye', how='left',  suffixes=("_icd10_g_df", "_icd10_asso_g_df"))


icd10_and_icd10_asso_merged_df


,RID_icd10_g_df,RCT_icd10_g_df,RMT_icd10_g_df,RCB_icd10_g_df,RMB_icd10_g_df,ID,URI,Name,Description,Synonyms,ICD10,RID_icd10_asso_g_df,RCT_icd10_asso_g_df,RMT_icd10_asso_g_df,RCB_icd10_asso_g_df,RMB_icd10_asso_g_df,Clinical_Records,ICD10_Eye
0,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130,2-86H4,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7Y2E,2-7XHP
1,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130,2-86MR,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7Y5R,2-7XHP
2,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130,2-86MW,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7Y5W,2-7XHP
3,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130,2-8764,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7YNP,2-7XHP
4,2-7XHP,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XHP,/id/2-7XHP,H40.1130,"Primary open-angle glaucoma, bilateral, stage ...",{},H40.1130,2-877J,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7YQE,2-7XHP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,2-7XK0,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XK0,/id/2-7XK0,H40.1233,"Low-tension glaucoma, bilateral, severe stage",{},H40.1233,2-8QV2,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8650,2-7XK0
1298,2-7XK2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XK2,/id/2-7XK2,H40.1234,"Low-tension glaucoma, bilateral, indeterminate...",{},H40.1234,2-8BGY,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-82WJ,2-7XK2
1299,2-7XK2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,eye-ai:2-7XK2,/id/2-7XK2,H40.1234,"Low-tension glaucoma, bilateral, indeterminate...",{},H40.1234,2-8BH2,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-82WP,2-7XK2
1300,2-7XK2,2024-04-15 23:51:21.917904+00,2024-04-15 23:51:21.917904+00,https://auth.globus.o

In [210]:

# Merging the dataframes based on Clinical_ID and Clinical_Records
final_merged_df_g = pd.merge(subject_obs_clinic_data_g, icd10_and_icd10_asso_merged_df, 
                           left_on='RID_clinic_df', right_on='Clinical_Records', how='inner')

# Display the result
# print(final_merged_df)

final_merged_df_g


,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Description,Synonyms,ICD10,RID_icd10_asso_g_df,RCT_icd10_asso_g_df,RMT_icd10_asso_g_df,RCB_icd10_asso_g_df,RMB_icd10_asso_g_df,Clinical_Records_y,ICD10_Eye
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132,2-8F4E,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT4,2-7XHT
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132,2-8696,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7XHT
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132,2-8F4J,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT8,2-7XHT
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132,2-869A,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7XHT
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, modera...",{},H40.1132,2-8F4P,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTC,2-7XHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2-7P5E,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1196,NaN,NaN,NaN,NaN,...,"Primary open-angle glaucoma, bilateral, severe...",{},H40.1133,2-8F3A,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868C,2-7XHW
1424,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,"Low-tension glaucoma, bilateral, mild stage",{},H40.1231,2-8QZ8,2024-04-15 23:56:35.278962+00,2024-04-15 23:56:35.278962+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868J,2-7XJW
1425,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,"Low-tension glaucoma, bilateral, mild stage",{},H40.1231,2-8F3M,2024-04-15 23:56:33.058909+00,2024-04-15 23:56:33.058909+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868M,2-7XJW
1426,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063

In [211]:


final_merged_df_g.Clinical_Records_y.nunique()


1236

In [212]:


final_merged_df_g.Subject_ID

0          1
1          1
2          1
3          1
4          2
        ... 
1423    1196
1424    1198
1425    1198
1426    1198
1427    1198
Name: Subject_ID, Length: 1428, dtype: int64

In [213]:
final_merged_df_g.to_csv("final_merged_df_g.csv", index=False)

In [ ]:
# # @title Execute Training algorithm
# from eye_ai.models.vgg19_hyper_parameter_tuning import main #import the new logistic module.
# with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
#   main()


In [ ]:
# # @title Save Execution Assets (model) and Metadata
# uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

# 